# **Bioinformatics with Jupyter Notebooks for WormBase:**
## **Data Retrieval 2 - Getting data from WormMine**
Welcome to the second jupyter notebook in the WormBase tutorial series. Over this series of tutorials, we will write code in Python that allows us to retrieve and perform simple analyses with data available on the WormBase sites.

This tutorial will deal with the WormBase data from WormMine. (http://intermine.wormbase.org/tools/wormmine/begin.do)
We will both explore the site, and the intermine python package, and extract data of interest. Let's get started!

We start by installing and loading the libraries that are required for this tutorial. 

In [ ]:
!pip install intermine
import intermine
from intermine import registry
from intermine.webservice import Service
import pandas as pd

To start, we use `getInfo(mine)` to fetch all the information about this particular mine i.e., its description, version, organisms associated etc.

In [ ]:
registry.getInfo("WormMine")

`getData(mine)` can be used to extract the corresponding data sets.
Note that this is not completely representative of the data available on WormMine. Check the WormMine site for the complete data.

In [ ]:
registry.getData("WormMine")

The method _new_query_ from Service class creates a query object

In [ ]:
service = Service("http://intermine.wormbase.org/tools/wormmine/service")
query=service.new_query()

### Performing simple queries on the WormMine database

Let's query the WormMine database to extract the commonName, genus, name, shortName, species, and taxonID of all organisms whose data is available:

In [ ]:
query=service.new_query("Organism")
query.select("commonName", "genus", "name", "shortName", "species","taxonId")

#Inserting first 10 rows of the query results into a dataframe and display the output!
organisms_data = pd.DataFrame(columns = ["commonName", "genus", "name", "shortName", "species","taxonId"])
for row in query.rows(start=0,size=10):
    info = {'commonName':row[0], 'genus':row[1], 'name':row[2], 'shortName':row[3], 'species':row[4], 
            'taxonId':row[5]}
    organisms_data = organisms_data.append(info, ignore_index = True)

organisms_data

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available:

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["automatedDescription", "biotype", "briefDescription", "length", "operon", 
                                     "primaryIdentifier", "secondaryIdentifier", "symbol"])

for row in query.rows(start=0,size=10):
    info = {'automatedDescription':row[0], 'biotype':row[1], 'briefDescription':row[2], 'length':row[3], 
            'operon':row[4], 'primaryIdentifier':row[5], 'secondaryIdentifier':row[6], 'symbol':row[7]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

Let's create a query object to query the description of all Gene Ontology (GO) terms from the WormMine database. We can then add extra columns (fields) to our query based on our needs - here, the identifier for all GO terms is added to the query. 
When there are multiple fields in a query, the default ordering of the output is based on the first field, but this can be changed to any other field.

In [ ]:
query=service.new_query()
query.select("GOTerm.description")

query.add_view("GOTerm.identifier") #Add a column to the query
 
query.add_sort_order("GOTerm.identifier") #Changing the sorting order of the query by a specific column

#Insert first 10 rows of the query results into a dataframe and display the output!
GO_data = pd.DataFrame(columns = ["GOTerm.description", "GOTerm.identifier"])
for row in query.rows(start=0,size=10):
    info = {'GOTerm.description':row[0], 'GOTerm.identifier':row[1]}
    GO_data = GO_data.append(info, ignore_index = True)

GO_data

### Performing queries on the WormMine database using constraints

Let's query the WormMine database to extract all details of all organisms for which data is available. Then we can add a constraint to the query to restrict outputs to only those with a certain value in one of the fields, here Caenorhabditis in genus.

In [ ]:
query=service.new_query("Organism")
query.select("commonName", "genus", "name", "shortName", "species","taxonId")

query.add_constraint("genus", "=", "Caenorhabditis") #Add a constraint to the query based on a column

#Insert first 10 rows of the query results into a dataframe and display the output!
organisms_data = pd.DataFrame(columns = ["commonName", "genus", "name", "shortName", "species","taxonId"])
for row in query.rows(start=0,size=10):
    info = {'commonName':row[0], 'genus':row[1], 'name':row[2], 'shortName':row[3], 'species':row[4], 
            'taxonId':row[5]}
    organisms_data = organisms_data.append(info, ignore_index = True)

organisms_data

Let's query the WormMine database to extract some details of all genes for which data is available. Then we can add multiple constraints. It is not necessary that the constraints are related to the fields explicitly mentioned in the query. 

Here, we add a constraint based on the value of genus being Caenorhabditis and another based on the value of ontologyTerm being kinase activity. 

(Even though the genus field is used as a constraint, the query does not return the column of genus as it has not been called in the query.)

In [ ]:
query=service.new_query("Gene")
query.select("primaryIdentifier", "ontologyAnnotations.id", "ontologyAnnotations.qualifier")

query.add_constraint("organism.genus", "=", "Caenorhabditis") #Add a constraint to the query based on a column

query.add_constraint("ontologyAnnotations.ontologyTerm.name", "=", "kinase activity") #Add a second constraint

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["primaryIdentifier", "ontologyAnnotations.id", 
                                     "ontologyAnnotations.qualifier"])
for row in query.rows(start=0,size=10):
    info = {'primaryIdentifier':row[0], 'ontologyAnnotations.id':row[1], 'ontologyAnnotations.qualifier':row[2]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

Let's query the WormMine database to extract all homologue data that is available along with the gene primaryIdentifiers and symbols for this data. Like mentiones previously, we can add multiple constraints. But it is not necessary that the constraints are applied one on the other.

We can use logic operators to apply constraints on the query outputs. `&` - `AND` and `|` - `OR` can be used in the regular sense.

Here, we have 3 constraints based on - the value of genus being Caenorhabditis, the value of species being elegans, and type of homologue being orthologue.

Then we apply the constraints on the query such that the output either has to have Caenorhabditis as genus AND elegans as species OR type of homologue as orthologue.

In [ ]:
query=service.new_query("Homologue")
query.select('Homologue.id', 'Homologue.type', 'gene.primaryIdentifier', 'gene.symbol')

query.add_constraint("gene.organism.genus", "=", "Caenorhabditis") #Set up constraint A
query.add_constraint("gene.organism.species", "=", "elegans") #Set up constraint B
query.add_constraint("type", "=", "orthologue") #Set up constraint C

query.set_logic("A & B & C") #Logic operators can be used to set the different constraints on the query

#Insert first 10 rows of the query results into a dataframe and display the output!
homologue_data = pd.DataFrame(columns = ["Homologue.id", "Homologue.type", "gene.primaryIdentifier", 
                                         "gene.symbol"])
for row in query.rows(start=0,size=10):
    info = {'Homologue.id':row[0], 'Homologue.type':row[1], 'gene.primaryIdentifier':row[2], 'gene.symbol':row[3]}
    homologue_data = homologue_data.append(info, ignore_index = True)

homologue_data

#### Different types of constraints

There are several kinds of constraints - Unary, Binary, Ternary, Multi-value, and List. We explore examples for all these constraint types.

##### Unary Constraints
Constraints that do not take any particular value but can be used to check if particular attirbute is absent or present.

Types of Unary Constraints - `IS Null` and `IS NOT Null`

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available, and then retain only those results where there is a valid secondaryIdentifier!

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

query.add_constraint("secondaryIdentifier","IS NOT NULL") #Unary Constraint

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["automatedDescription", "biotype", "briefDescription", "length", "operon", 
                                     "primaryIdentifier", "secondaryIdentifier", "symbol"])

for row in query.rows(start=0,size=10):
    info = {'automatedDescription':row[0], 'biotype':row[1], 'briefDescription':row[2], 'length':row[3], 
            'operon':row[4], 'primaryIdentifier':row[5], 'secondaryIdentifier':row[6], 'symbol':row[7]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

##### Binary Constraints
Constraints that can take a particular attribute which can then be compared them to a specified value.

Types of Binary Constraints - `=`, `<=`, `>=`, `<`, `>`, `!=`.

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available, and then retain only those results where the value of the length is greater than or equal to 12000!

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

query.add_constraint("length", ">=", "12000") #Binary Constraint

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["automatedDescription", "biotype", "briefDescription", "length", "operon", 
                                     "primaryIdentifier", "secondaryIdentifier", "symbol"])

for row in query.rows(start=0,size=10):
    info = {'automatedDescription':row[0], 'biotype':row[1], 'briefDescription':row[2], 'length':row[3], 
            'operon':row[4], 'primaryIdentifier':row[5], 'secondaryIdentifier':row[6], 'symbol':row[7]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

##### Ternary constraints
Constraints that have one required value and one optional value.

Types of Ternary Constraints `-LOOKUP` (this operator searches through all the fields in a particular class for the value specified by the user)

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available, and then retain only those results where there is a mention of 'GABA' in any field!

The extra_value parameter can be used to limit the search to the type of object (for example, organism in genes), here C. elegans.

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

query.add_constraint("Gene", "LOOKUP", "hlh-2", extra_value='C. elegans') #Ternary Constraint

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["automatedDescription", "biotype", "briefDescription", "length", "operon", 
                                     "primaryIdentifier", "secondaryIdentifier", "symbol"])

for row in query.rows(start=0,size=10):
    info = {'automatedDescription':row[0], 'biotype':row[1], 'briefDescription':row[2], 'length':row[3], 
            'operon':row[4], 'primaryIdentifier':row[5], 'secondaryIdentifier':row[6], 'symbol':row[7]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

##### Multi-value Constraints - 
Constraints that can take multiple values.

Types of Multi-value Constraints `-ONE` OF and `NONE OF`

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available, and then retain only those results where the gene symbol is one of hlh-2, unc-26, gar-3, or gbb-2.

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

query.add_constraint("symbol","ONE OF",['hlh-2','unc-26', 'gar-3', 'gbb-2']) #Multi-value Constraint

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["automatedDescription", "biotype", "briefDescription", "length", "operon", 
                                     "primaryIdentifier", "secondaryIdentifier", "symbol"])

for row in query.rows(start=0,size=10):
    info = {'automatedDescription':row[0], 'biotype':row[1], 'briefDescription':row[2], 'length':row[3], 
            'operon':row[4], 'primaryIdentifier':row[5], 'secondaryIdentifier':row[6], 'symbol':row[7]}
    genes_data = genes_data.append(info, ignore_index = True)
genes_data

##### List constraints 
Constraints that contain a list of values.

Types of List Constraints `-IN` or `NOT IN` 

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available, and then retain only those results where the Gene is in the publicly available list C. elegans transcription factor genes present on WormMine.

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

query.add_constraint("Gene", "IN", "C. elegans transcription factor genes") #List Constraint

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["automatedDescription", "biotype", "briefDescription", "length", "operon", 
                                     "primaryIdentifier", "secondaryIdentifier", "symbol"])

for row in query.rows(start=0,size=10):
    info = {'automatedDescription':row[0], 'biotype':row[1], 'briefDescription':row[2], 'length':row[3], 
            'operon':row[4], 'primaryIdentifier':row[5], 'secondaryIdentifier':row[6], 'symbol':row[7]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

##### Sub-Class constraints
Constraints that allow a user to specify a sub-class of a class to constrain a path to

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available, and then constrain our results to only those items of the sub class GOAnnotation of ontologyAnnotations class.

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

query.add_constraint("ontologyAnnotations", "GOAnnotation") #Sub-Class constraint

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["automatedDescription", "biotype", "briefDescription", "length", "operon", 
                                     "primaryIdentifier", "secondaryIdentifier", "symbol"])
for row in query.rows(start=0,size=10):
    info = {'automatedDescription':row[0], 'biotype':row[1], 'briefDescription':row[2], 'length':row[3], 
            'operon':row[4], 'primaryIdentifier':row[5], 'secondaryIdentifier':row[6], 'symbol':row[7]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

##### Loop Constraints - 
Constraints that assert that two paths refer to the same object

Types of Loop Constraints - `IS` or `IS NOT`

Let's query the WormMine database to extract the automatedDescription, biotype, briefDescription, length, operon, primaryIdentifier, secondaryIdentifier, and symbol of all genes whose data is available, constrain our results using a list constraint and then, a loop constraint.

In [ ]:
query=service.new_query("Gene")
query.select("automatedDescription", "biotype", "briefDescription", "length", "operon", "primaryIdentifier", 
             "secondaryIdentifier", "symbol")

query.add_view("homologues.gene.primaryIdentifier","homologues.homologue.primaryIdentifier") #Add more fields

query.add_constraint("Gene", "IN", "C. elegans transcription factor genes", code = "A") #List constraint

query.add_constraint("homologues.homologue", "IS NOT", "Gene", code = "B") #Loop Constraint


#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["homologues.gene.primaryIdentifier", "homologues.homologue.primaryIdentifier"])
for row in query.rows(start=0,size=10):
    info = {'homologues.gene.primaryIdentifier':row['homologues.gene.primaryIdentifier'], 
            'homologues.homologue.primaryIdentifier':row['homologues.homologue.primaryIdentifier']}
    genes_data = genes_data.append(info, ignore_index = True)
genes_data

##### Range Constraints - 
Constraints that test whether a value lies relative to a set of ranges or not

Types of Range Constraints - OVERLAPS, DOES NOT OVERLAP, WITHIN, OUTSIDE, CONTAINS and DOES NOT CONTAIN

Let's query the WormMine database to extract the details on organism name and chromosome location of all sequences whose data is available, and then constrain our results based on if the chromosome location overlaps our specified range of I:1..4000.

In [ ]:
query=service.new_query()

query.add_view("SequenceFeature.organism.shortName", 
               "SequenceFeature.chromosomeLocation.locatedOn.primaryIdentifier", 
               "SequenceFeature.chromosomeLocation.start", "SequenceFeature.chromosomeLocation.end" ) #Add fields

query.add_constraint("chromosomeLocation", "OVERLAPS", ["I:1..4000"]) #Range constraint



#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["Organism", "Chromosome identifier", "Chromosome Start Location", 
                                     "Chromosome End Location"])

for row in query.rows(start=0,size=10):
    info = {'Organism':row[0], 'Chromosome identifier':row[1], 'Chromosome Start Location':row[2], 
            'Chromosome End Location':row[3]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

### Some query examples and exploring some other functionalities

Let's query the WormMine database to extract the primaryIdentifier, symbol, of all genes whose data is available and connect that to the name and identifiers of the ontology terms. We then add a constraint that the homologue type of the results should be orthologue:

In [ ]:
query=service.new_query("Gene")
query.select("primaryIdentifier","symbol", "ontologyAnnotations.ontologyTerm.name", 
             "ontologyAnnotations.ontologyTerm.identifier")
query.add_constraint("homologues.type", "=", "orthologue")

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["primaryIdentifier", "symbol", "Ontology Name", "Ontology Identifier"])
for row in query.rows(start=0,size=10):
    info = {'primaryIdentifier':row[0], 'symbol':row[1], 'Ontology Name':row[2], 'Ontology Identifier':row[3]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

It is possible to perform INNER and OUTER joins on the queries to get columns from different sets of data easily!

In [ ]:
query.add_view('homologues.type')
query.add_join("homologues", "INNER")

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["primaryIdentifier", "symbol", "Ontology Name", "Ontology Identifier",
                                    "Homologue Type"])

for row in query.rows(start=0,size=10):
    info = {'primaryIdentifier':row[0], 'symbol':row[1], 'Ontology Name':row[2], 'Ontology Identifier':row[3], 
            'Homologue Type':row[4]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

##### Combinations of constraints and set logic

Let's query the organism name and gene symbol for all genes in the WormMine database. We then constrain our results based on the logic OR of two specified constraints as in the cell below:

In [ ]:
query = service.new_query()
query.add_view("Gene.organism.name","Gene.symbol")

gene_is_ugt = query.add_constraint("Gene.symbol", "=", "ugt-59") #Add first binary constraint
gene_is_sgn = query.add_constraint("Gene.symbol", "=", "sgn-1") #Add second binary constraint
query.set_logic(gene_is_ugt | gene_is_sgn) #Logic OR on the 2 constraints

#Insert first 10 rows of the query results into a dataframe and display the output!
genes_data = pd.DataFrame(columns = ["Gene.organism.name", "Gene.symbol"])
for row in query.rows(start=0,size=10):
    info = {'Gene.organism.name':row[0], 'Gene.symbol':row[1]}
    genes_data = genes_data.append(info, ignore_index = True)

genes_data

### Writing query results to a file for later use

Since we use dataframes for storing the data, we can easily choose any rows or columns we want to retain based on simple constraints using the various functionalities present in the pandas library!

In [ ]:
genes_data.to_csv('results.csv')

### Getting a readable XML serialisation of a query

In [ ]:
query.to_xml()

### Clearing the output of a query

In [ ]:
query.clear_view()

This is the end of the tutorial for querying and extracting WormBase data using WormMine through intermine. This tutorial is influenced by the intermine tutorial notebooks from - https://github.com/intermine/intermine-ws-python-docs

In the next tutorial, we will use access the WormBase ParaSite data through their RESTful API.